# Imports

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Time series
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

# ML
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.filterwarnings('ignore')

# Configurations
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Loading data

In [90]:
# Load sales and market data
sales_df = pd.read_csv('../Data/Case2_Sales data.csv', sep=";")
market_df = pd.read_excel('/Users/diogocarvalho/Documents/GitHub/Case-2--Siemens-Sales-Forecast/Data/Case2_Market data.xlsx', header=[0,1,2])

# Preprocessing sales data

> Sales_EUR is object and must be numeric

In [91]:
# fix the type of Sales_EUR column
sales_df['Sales_EUR'] = (
    sales_df['Sales_EUR']
    .replace(',','.', regex=True)        
    .replace(r'[^0-9\.\-]', '', regex=True) 
    .pipe(pd.to_numeric, errors='coerce')
)

In [92]:
sales_df['Sales_EUR'].dtype

dtype('float64')

> DATE is object, and must be converted to datetime

In [93]:
# Convert the DATE column to datetime
sales_df['DATE'] = pd.to_datetime(sales_df['DATE'], format='%d.%m.%Y')

In [94]:
# Aggregate to monthly level
sales_df['YearMonth'] = sales_df['DATE'].dt.to_period('M')
monthly_sales = sales_df.groupby(['YearMonth', 'Mapped_GCK'])['Sales_EUR'].sum().reset_index()

# Convert YearMonth to datetime
monthly_sales['YearMonth'] = monthly_sales['YearMonth'].dt.to_timestamp()

In [95]:
# make DATE index of sales_df
# sales_df = sales_df.set_index('DATE')

In [96]:
sales_df.head()

,DATE,Mapped_GCK,Sales_EUR,YearMonth
0,2018-10-01,#1,0.0,2018-10
1,2018-10-02,#1,0.0,2018-10
2,2018-10-03,#1,0.0,2018-10
3,2018-10-04,#1,0.0,2018-10
4,2018-10-05,#1,0.0,2018-10


> Aggregating sales per product based on month and year

Since we will be doing monthly analysis, it's no use for us to have the data in days.

In [97]:
# Aggregate sales data by month for each product
monthly_sales_data = sales_df.groupby(['YearMonth', 'Mapped_GCK'], as_index=False).agg({
    'Sales_EUR': 'sum'  # Sum sales revenue
})

In [98]:
# Display the first few rows to check the transformation
monthly_sales_data.head(25)

,YearMonth,Mapped_GCK,Sales_EUR
0,2018-10,#1,36098918.79
1,2018-10,#11,1021303.50
2,2018-10,#12,28686.33
3,2018-10,#13,27666.10
4,2018-10,#14,5770.00
5,2018-10,#16,333196.87
6,2018-10,#20,4563.14
7,2018-10,#3,8089465.96
8,2018-10,#36,6474.60
9,2018-10,#4,397760.69


In [99]:
monthly_sales_data.shape

(602, 3)

### Separate products into different datasets

We will predcit procuts individually, so separating them will be useful

In [100]:
# replace the "#" with "Product" in the product names
monthly_sales_data['Mapped_GCK'] = monthly_sales_data['Mapped_GCK'].str.replace('#', 'Product')

In [101]:
# Get the unique products
unique_products = monthly_sales_data['Mapped_GCK' ].unique()
unique_products

array(['Product1', 'Product11', 'Product12', 'Product13', 'Product14',
       'Product16', 'Product20', 'Product3', 'Product36', 'Product4',
       'Product5', 'Product6', 'Product8', 'Product9'], dtype=object)

In [102]:
# Create a dictionary to store the DataFrames
product_dataframes = {product: monthly_sales_data[monthly_sales_data['Mapped_GCK'] == product] for product in unique_products}

In [103]:
# Loop through each product and assign it to a dynamically named variable
for i, product in enumerate(product_dataframes.keys(), start=1):
    globals()[f"product{i}"] = product_dataframes[product]

In [104]:
# Check the shape of each product DataFrame
for i in range(1, len(product_dataframes) + 1):
    print(f"product{i} shape: {globals()[f'product{i}'].shape}")

product1 shape: (43, 3)
product2 shape: (43, 3)
product3 shape: (43, 3)
product4 shape: (43, 3)
product5 shape: (43, 3)
product6 shape: (43, 3)
product7 shape: (43, 3)
product8 shape: (43, 3)
product9 shape: (43, 3)
product10 shape: (43, 3)
product11 shape: (43, 3)
product12 shape: (43, 3)
product13 shape: (43, 3)
product14 shape: (43, 3)


In [105]:
product1.head()

,YearMonth,Mapped_GCK,Sales_EUR
0,2018-10,Product1,36098918.79
14,2018-11,Product1,5140760.00
28,2018-12,Product1,37889612.12
42,2019-01,Product1,27728148.35
56,2019-02,Product1,34793163.53


# Preprocessing market data

In [106]:
market_df.head()

Unnamed: 0_level_0  \
  Index 2010=100 (if not otherwise noted)   
                                     date   
0                                  2004m2   
1                                  2004m3   
2                                  2004m4   
3                                  2004m5   
4                                  2004m6   

                                     China  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO156   
0                                16.940704   
1                                23.711852   
2                                24.435235   
3                                23.708115   
4                                27.009138   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP156   
0                               16.940704   
1                               23.711852   
2                               24.435235   
3                               23.708115   
4                               27.009138   

                                    France  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO250   
0                               112.091273   
1                               136.327976   
2                               117.791806   
3                               109.002541   
4                               133.785737   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP250   
0                               83.458866   
1                              106.168192   
2                               92.007646   
3                               85.696486   
4                              106.641482   

                                   Germany  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO276   
0                                82.623037   
1                               100.556582   
2                                89.653203   
3                                86.880571   
4                                99.010814   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP276   
0                               79.452532   
1                               97.012918   
2                               84.932358   
3                               82.372794   
4                               95.108740   

                                     Italy  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO380   
0                               124.289603   
1                               143.411662   
2                               129.083828   
3                               135.590391   
4                               136.424935   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP380   
0                               86.560493   
1                              106.344544   
2                               95.579673   
3                              100.087039   
4                              110.889719   

                                     Japan  \
  Production Index Machinery & Electricals   
                            MAB_ELE_PRO392   
0                               109.334010   
1                               140.884616   
2                               105.853579   
3                               101.864777   
4                               120.332920   

                                           \
  Shipments Index Machinery & Electricals   
                           MAB_ELE_SHP392   
0                              110.495272   
1                              144.686166   
2                              102.655769   
3                              100.305285   
4                              119.616380   

            

In [107]:
# drop the second header row
market_df.columns = market_df.columns.droplevel(1)

In [108]:
# Join the header levels
market_df.columns = [
    '_'.join([str(level) for level in col_tuple]).strip()
    for col_tuple in market_df.columns
]


In [109]:
# change first column name to "DATE"

market_df.rename(columns={'Unnamed: 0_level_0_date': 'DATE'}, inplace=True)

In [110]:
# Replace the "m" with "-" in DATE column of market_df
market_df['DATE'] = market_df['DATE'].str.replace('m', '-')

In [111]:
# Convert the DATE column to datetime
market_df['DATE'] = pd.to_datetime(market_df['DATE'], infer_datetime_format=True)

In [112]:
# Create the YearMonth column, like done before in the sales_df
market_df['YearMonth'] = market_df['DATE'].dt.to_period('M')

# Merge market data

In [113]:
# Ensure macro data is monthly
market_df['YearMonth'] = market_df['DATE'].dt.to_period('M').dt.to_timestamp()

# Merge with sales
full_df = pd.merge(monthly_sales, market_df, on='YearMonth', how='left')

# Sort for time series modeling
full_df = full_df.sort_values(['Mapped_GCK', 'YearMonth']).reset_index(drop=True)

In [114]:
full_df.tail()

,YearMonth,Mapped_GCK,Sales_EUR,DATE,China_MAB_ELE_PRO156,China_MAB_ELE_SHP156,France_MAB_ELE_PRO250,France_MAB_ELE_SHP250,Germany_MAB_ELE_PRO276,Germany_MAB_ELE_SHP276,Italy_MAB_ELE_PRO380,Italy_MAB_ELE_SHP380,Japan_MAB_ELE_PRO392,Japan_MAB_ELE_SHP392,Switzerland_MAB_ELE_PRO756,Switzerland_MAB_ELE_SHP756,United Kingdom_MAB_ELE_PRO826,United Kingdom_MAB_ELE_SHP826,United States_MAB_ELE_PRO840,United States_MAB_ELE_SHP840,Europe_MAB_ELE_PRO1100,Europe_MAB_ELE_SHP1100,Europe_RohiBASEMET1000_org,Europe_RohiENERGY1000_org,Europe_RohiMETMIN1000_org,Europe_RohiNATGAS1000_org,Europe_RohCRUDE_PETRO1000_org,Europe_RohCOPPER1000_org,Europe_WKLWEUR840_org,Producer Prices_PRI27840_org,Producer Prices_PRI27826_org,Producer Prices_PRI27380_org,Producer Prices_PRI27250_org,Producer Prices_PRI27276_org,Producer Prices_PRI27156_org,production index_PRO28840_org,production index_PRO281000_org,production index_PRO28756_org,production index_PRO28826_org,production index_PRO28380_org,production index_PRO28392_org,production index_PRO28250_org,production index_PRO28276_org,production index_PRO27840_org,production index_PRO271000_org,production index_PRO27756_org,production index_PRO27826_org,production index_PRO27380_org,production index_PRO27392_org,production index_PRO27250_org,production index_PRO27276_org
597,2021-12-01,#9,14935.2,2021-12-01,310.763183,310.763183,100.565744,134.589504,118.103281,149.364286,94.006826,150.482735,127.771735,131.029703,106.704029,104.819189,101.273544,NaN,107.040766,148.590371,123.076659,150.046922,125.207030,112.372958,116.715183,236.488368,92.188708,126.761240,1.1304,128.511261,NaN,113.309631,108.182510,115.748863,98.106200,105.736748,134.598755,102.277530,90.350055,103.191399,136.975506,112.791885,129.188248,109.624107,132.281006,114.326241,121.065762,72.915611,109.005151,80.763306,97.773956
598,2022-01-01,#9,2064.7,2022-01-01,235.956129,235.956129,85.743503,108.156320,94.550610,120.353403,86.851008,101.258277,110.460181,110.823532,103.499260,101.701570,95.003541,NaN,111.052133,129.565798,103.199827,120.338095,133.219393,121.309886,125.229641,196.911140,106.173052,129.829146,1.1314,131.628510,NaN,115.390617,111.037476,117.853386,98.280171,110.894371,117.489883,100.305236,85.444170,92.292313,117.861377,90.558372,92.343117,111.364670,122.236023,108.999212,112.324119,74.355736,95.369065,77.944954,98.599052
599,2022-02-01,#9,901.5,2022-02-01,235.956129,235.956129,90.603540,117.715770,103.987916,129.383676,106.583758,120.956538,117.879631,118.300232,100.294492,98.583952,98.458412,NaN,116.336327,138.560330,113.500635,131.500126,138.905572,131.273215,131.176501,197.523679,118.348203,131.963648,1.1342,133.342178,NaN,116.431107,112.057098,118.905647,98.714158,117.168167,124.627762,98.332942,89.021378,113.290565,124.710859,97.766502,102.820961,114.688400,127.373421,103.672183,115.557330,91.182419,103.950687,79.001831,106.128059
600,2022-03-01,#9,4452.6,2022-03-01,329.413367,329.413367,107.843548,136.858720,121.308119,151.201314,124.637966,153.645142,152.000561,156.400634,97.089723,95.466333,121.993915,NaN,117.654038,165.926217,133.133010,158.055622,149.890871,163.186834,141.283339,271.079906,142.200872,135.782207,1.1019,136.153778,NaN,117.471596,112.362991,119.852684,99.021554,118.910912,149.375229,96.360648,109.155949,134.288818,160.954233,114.720810,122.049515,115.164093,152.452942,98.345154,145.254965,102.475998,133.743932,96.704582,119.948433
601,2022-04-01,#9,3285.0,2022-04-01,267.373145,267.373145,87.698110,116.528738,99.522205,127.022869,103.556690,128.733305,114.262328,115.012049,NaN,NaN,95.266502,NaN,116.961047,NaN,112.902215,134.935510,146.090998,153.188945,138.094143,243.436030,130.835430,134.859685,1.0819,137.531616,NaN,118.408043,113.280655,121.220627,98.857087,119.385483,128.285706,NaN,84.728728,111.090744,120.098810,91.979698,98.675873,112.158089,134.843353,NaN,114.359844,86.255684,102.361680,80.763306,101.074341


# Feature Engineering

`Lag features` answer the question "what were the sales X months ago?" - we are using lag with X=[1,3,12] (1 month, 1 trimester, 1 year)

`Rolling mean(moving average)`capture short-term trends or smooths out noise. For example, April 2021, it averages Jan–Mar 2021 (but shifted 1 step so we're not using the current month’s value).

In [115]:
# Example: Add lag and rolling mean for each product group
for lag in [1, 3, 12]:
    full_df[f'lag_{lag}'] = full_df.groupby('Mapped_GCK')['Sales_EUR'].shift(lag)

for window in [3, 6]:
    full_df[f'rolling_mean_{window}'] = full_df.groupby('Mapped_GCK')['Sales_EUR'].transform(lambda x: x.shift(1).rolling(window).mean())

# Model with XGBoost (ML Approach)

In [116]:
results = []

# Loop through each product
for gck in full_df['Mapped_GCK'].unique():
    gck_df = full_df[full_df['Mapped_GCK'] == gck].dropna()
    
    train = gck_df[gck_df['YearMonth'] < '2022-05-01']
    test = gck_df[gck_df['YearMonth'] >= '2022-05-01']
    
    features = [col for col in gck_df.columns if col not in ['Sales_EUR', 'Date', 'YearMonth', 'Mapped_GCK']]
    
    model = XGBRegressor(n_estimators=100, learning_rate=0.1)
    model.fit(train[features], train['Sales_EUR'])
    
    preds = model.predict(test[features])
    
    rmse = np.sqrt(mean_squared_error(test['Sales_EUR'], preds))
    results.append((gck, rmse))
    
    # Save predictions for submission
    test['Predicted_Sales'] = preds
    test[['YearMonth', 'Mapped_GCK', 'Predicted_Sales']].to_csv(f'predictions_{gck}.csv', index=False)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:DATE: datetime64[ns]

# Model with SARIMAX (Statistical Approach)

In [ ]:
# Example for 1 product
gck = 'Product_X'
gck_df = full_df[full_df['Mapped_GCK'] == gck].set_index('YearMonth')

endog = gck_df['Sales_EUR']
exog = gck_df[market_df.columns.difference(['Date', 'YearMonth'])]

# Train/test split
train_endog = endog[:'2022-04']
test_endog = endog['2022-05':]

train_exog = exog.loc[:'2022-04']
test_exog = exog.loc['2022-05':]

model = SARIMAX(train_endog, exog=train_exog, order=(1,1,1), seasonal_order=(1,1,1,12))
sarimax_res = model.fit()

forecast = sarimax_res.predict(start=test_endog.index[0], end=test_endog.index[-1], exog=test_exog)
rmse = np.sqrt(mean_squared_error(test_endog, forecast))
print(f"RMSE for {gck}: {rmse}")

# Submission Formatting

In [ ]:
# Combine all test predictions into one file
final_submission = pd.concat([pd.read_csv(f'predictions_{gck}.csv') for gck in full_df['Mapped_GCK'].unique()])
final_submission.columns = ['Year Month', 'Mapped_GCK', 'Sales EUR']
final_submission.to_csv("Submission_Template.csv", index=False)